<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Grouping-in-SQL" data-toc-modified-id="Grouping-in-SQL-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Grouping in SQL</a></span><ul class="toc-item"><li><span><a href="#Grouping-statements" data-toc-modified-id="Grouping-statements-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Grouping statements</a></span><ul class="toc-item"><li><span><a href="#Task" data-toc-modified-id="Task-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Task</a></span><ul class="toc-item"><li><span><a href="#Possible-Solution" data-toc-modified-id="Possible-Solution-2.1.1.1"><span class="toc-item-num">2.1.1.1&nbsp;&nbsp;</span>Possible Solution</a></span></li></ul></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Exercise</a></span></li><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li></ul></li><li><span><a href="#Joins" data-toc-modified-id="Joins-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Joins</a></span></li></ul></div>

![sql](sql-logo.jpg)

In [ ]:
import pandas as pd
import sqlite3
import pandasql

conn = sqlite3.connect("flights.db")
cur = conn.cursor()

# Objectives

- Use SQL aggregation functions with GROUP BY
- Use HAVING for group filtering
- Use SQL JOIN to combine tables using keys

# Grouping in SQL

## Grouping statements

Combine `SELECT` and `GROUP BY` when you want *aggregates* by values

`SELECT` `COUNT`, `MIN(x)`, `MAX(x)`, `SUM(x)`, etc.

`GROUP BY x`

### Task

- Which countries have the highest numbers of active airlines? Return the top 25.

#### Possible Solution

In [ ]:
pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
    LIMIT 25
''', conn)

### Exercise

- Which countries have the highest numbers of inactive airlines? Return all the countries that have more than 10.

In [ ]:
# Your code here


### Exercise

- Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [ ]:
# Your code here


# Joins